In [1]:
import pandas as pd
from selenium.webdriver import Chrome
from selenium import webdriver
from time import sleep
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys

In [2]:
publishNews = pd.read_excel('df_skoob.xlsx')

In [3]:
publishNews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10188 entries, 0 to 10187
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    10188 non-null  int64 
 1   Unnamed: 0.1  10188 non-null  int64 
 2   titulo        10188 non-null  object
 3   autor         10188 non-null  object
 4   editora       10188 non-null  object
 5   categoria     10188 non-null  object
 6   paginas       10188 non-null  object
 7   isbn          10188 non-null  int64 
 8   nota          10188 non-null  object
 9   avaliacoes    10188 non-null  object
dtypes: int64(3), object(7)
memory usage: 796.1+ KB


In [4]:
#filtrar apenas as editoras com as quais eu irei trabalhar para saber se no final as informações estarão certas.
editoras = publishNews[publishNews['editora'].isin(['Gente','Todavia','L&PM POCKET','Novo Século','Academia','LeYa',
                                                    'Astral Cultural','Ática','On Line Editora','Pão Diário',
                                                    'WMF Martins Fontes','Figurati','Martins Fontes','Verus','Cultrix',
                                                    'Quatro Ventos','Agir','Elsevier','Gutenberg','Editora Vida',
                                                    'Cia das Letrinhas','Escrituras','Coquetel','Thomas Nelson Brasil'])]

In [6]:
#ativando chromedrive:
navegador = webdriver.Chrome('/Users/perolatai/Downloads/chromedriver')

### aqui vou criar as funções que utilizarei abaixo

In [7]:
#função para filtrar a editora
def filtrarEditora(nomeEditora):
    return editoras[editoras['editora'].isin([nomeEditora])]

In [8]:
#função para verificar o número de duplicados
def verificarDuplicados(dataframe, coluna):
    #total de itens duplicados
    print('Total de itens duplicados: ', len(dataframe[coluna])-len(dataframe[coluna].drop_duplicates()))
    #total de itens não duplicados
    print('Total de itens não duplicados: ', len(dataframe[coluna].drop_duplicates()))

In [9]:
#função para filtrar apenas 1 resultado dos valores duplicados
def duplicados(dataframe, coluna):
    #filtrando apenas o isbn
    return dataframe[coluna].drop_duplicates()

In [10]:
#transformando o dicionário em dataframe
def dic_dataframe(dicionario, nomeColuna):
    return pd.DataFrame.from_dict(data=dicionario, orient='index', columns=[nomeColuna])

In [11]:
#função pra resetar o indice:
def resetaIndex(dataframe):
    return dataframe.reset_index()

In [12]:
#renomeando colunas:
def renomeiaColuna(dataframe, nomeColuna):
    dataframe.columns = ['isbn', nomeColuna]

### Editora Gente

In [13]:
#filtrando a editora
editoraGente = filtrarEditora('Gente')

In [14]:
#verificando os valores duplicados
verificarDuplicados(editoraGente, 'isbn')

Total de itens duplicados:  429
Total de itens não duplicados:  102


In [15]:
#filtrando isbn não duplicados
edGente = duplicados(editoraGente, 'isbn')

In [49]:
#buscando as informações
preco = {}
gente = 'https://www.editoragente.com.br/'
#barra de progresso
with tqdm(total=len(edGente)) as pbar:
    for item in edGente:
        pbar.update(1)
        try:
            navegador.get(gente)
            sleep(1)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//div[@class="nav-input UI-NAV-INPUT"]').click()
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@class="input-text UI-SEARCH"]')
            digitaCodigo.send_keys(item)
            #clicar em pesquisar
            navegador.find_element_by_xpath('//div[@class="nav-submit-button"]').click()
            #pegar preço
            if navegador.find_element_by_xpath('//span[@class="regular-price"]').text == '':
                preco[item] = navegador.find_element_by_xpath('//*[@id="old-price-348"]').text
            else:
                preco[item] = navegador.find_element_by_xpath('//span[@class="regular-price"]').text
            
            
            #valor = navegador.find_element_by_xpath('//span[@class="regular-price"]')
            #preco[item] = valor.text
            
        except:
            preco[item] = 'Não Encontrado'

In [51]:
#transformando o dicionário em dataframe
dfGente = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfGente = resetaIndex(dfGente)
#renomeando colunas
renomeiaColuna(dfGente, 'preco')

### Todavia

In [53]:
#filtrando a editora
editoraTodavia = filtrarEditora('Todavia')

In [54]:
#verificando os valores duplicados
verificarDuplicados(editoraTodavia, 'isbn')

Total de itens duplicados:  24
Total de itens não duplicados:  2


In [55]:
#filtrando isbn não duplicados
edTodavia = duplicados(editoraTodavia, 'isbn')

In [56]:
#buscando as informações
preco = {}
todavia = 'https://todavialivros.com.br/'
#barra de progresso
with tqdm(total=len(edTodavia)) as pbar:
    for item in edTodavia:
        pbar.update(1)
        try:
            navegador.get(todavia)
            sleep(1)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//a[@id="bt-search"]').click()
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//*[@id="input-search"]')
            digitaCodigo.send_keys(item)
            #clicar em pesquisar
            navegador.find_element_by_xpath('//button[@class="btn btn-default btn-search"]').click()
            #entrar nos detalhes
            navegador.find_element_by_xpath('//*[@id="search-results"]/div/div/div/div/article/a/figure').click()
            #pegar preço
            valor = navegador.find_element_by_xpath('//span[@class="price "]')
            preco[item] = valor.text
            
        except:
            preco[item]  = 'Não Encontrado'

In [57]:
#transformando o dicionário em dataframe
dftodavia = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dftodavia = resetaIndex(dftodavia)
#renomeando colunas
renomeiaColuna(dftodavia, 'preco')

### Novo Século

In [59]:
#filtrando a editora
editoraNovoseculo = filtrarEditora('Novo Século')

In [60]:
#verificando os valores duplicados
verificarDuplicados(editoraNovoseculo, 'isbn')

Total de itens duplicados:  7
Total de itens não duplicados:  8


In [61]:
#filtrando isbn não duplicados
edNovoseculo = duplicados(editoraNovoseculo, 'isbn')

In [71]:
#buscando as informações
preco = {}
novoSeculo = 'https://www.gruponovoseculo.com.br/'
#barra de progresso
with tqdm(total=len(edNovoseculo)) as pbar:
    for item in edNovoseculo:
        pbar.update(1)
        try:
            navegador.get(novoSeculo)
            sleep(2)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//div[@class="box only-responsive"]').click()
            sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//*[@id="buscar"]')
            digitaCodigo.send_keys(item)
            #clicar em pesquisar
            navegador.find_element_by_xpath('//button[@class="fas fa-search"]').click()
            sleep(1)
            #entrar nos detalhes
            navegador.find_element_by_xpath('//*[@id="app"]/main/section/div[2]/div[2]/div/article/a[1]').click()
            sleep(1)
            #pegar preço
            valor = navegador.find_element_by_xpath('//div[@class="valor"]')
            preco[item] = valor.text

        except:
            preco[item]      = 'Não Encontrado'

In [73]:
#transformando o dicionário em dataframe
dfnovoSeculo = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfnovoSeculo = resetaIndex(dfnovoSeculo)
#renomeando colunas
renomeiaColuna(dfnovoSeculo, 'preco')

### Academia

In [75]:
#filtrando a editora
editoraAcademia = filtrarEditora('Academia')

In [76]:
#verificando os valores duplicados
verificarDuplicados(editoraAcademia, 'isbn')

Total de itens duplicados:  61
Total de itens não duplicados:  15


In [77]:
#filtrando isbn não duplicados
edAcademia = duplicados(editoraAcademia, 'isbn')

In [78]:
#buscando as informações
preco = {}
academia = 'https://www.planetadelivros.com.br/editorial/academia/326'
#barra de progresso
with tqdm(total=len(edAcademia)) as pbar:
    for item in edAcademia:
        pbar.update(1)
        try:
            navegador.get(academia)
            sleep(1)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//input[@class="megacercador-casella ui-autocomplete-input"]').click()
            #sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@class="megacercador-casella ui-autocomplete-input"]')
            digitaCodigo.send_keys(item)
            #clicar em pesquisar
            navegador.find_element_by_xpath('//button[@class="megacercador-enviar btn"]').click()
            #entrar nos detalhes
            navegador.find_element_by_xpath('//*[@id="resultat-llibres"]/ul/li/div[1]/a').click()
            sleep(2)
            #pegar preço
            valor = navegador.find_element_by_xpath('//*[@id="soportes-desktop"]/div/div[3]/div/label/div/div')
            preco[item] = valor.text

        except:
            preco[item]      = 'Não Encontrado'

In [80]:
#transformando o dicionário em dataframe
dfacademia = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfacademia = resetaIndex(dfacademia)
#renomeando colunas
renomeiaColuna(dfacademia, 'preco')

### Astral Cultural

In [82]:
#filtrando a editora
editoraAstralCultural = filtrarEditora('Astral Cultural')

In [83]:
#verificando os valores duplicados
verificarDuplicados(editoraAstralCultural, 'isbn')

Total de itens duplicados:  71
Total de itens não duplicados:  17


In [84]:
#filtrando isbn não duplicados
edAstralCultural = duplicados(editoraAstralCultural, 'isbn')

In [85]:
#buscando as informações
preco = {}
astralCultural = 'https://loja.astralcultural.com.br/'
#barra de progresso
with tqdm(total=len(edAstralCultural)) as pbar:
    for item in edAstralCultural:
        pbar.update(1)
        try:
            navegador.get(astralCultural)
            sleep(1)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//input[@class="form-control search-input"]').click()
            #sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@class="form-control search-input"]')
            digitaCodigo.send_keys(item)
            #clicar em pesquisar
            navegador.find_element_by_xpath('//button[@type="submit"]').click()
            #entrar nos detalhes
            navegador.find_element_by_xpath('//*[@id="variant-list"]/div/div/div/div[5]/div[2]/div[1]/div/article/figure/a/img').click()
            sleep(2)
            #pegar preço
            valor = navegador.find_element_by_xpath('/html/body/section/div/div/div[2]/div/div[2]/div[2]/div/p/span/del')
            preco[item] = valor.text
            
        except:
            preco[item]      = 'Não Encontrado'


In [87]:
#transformando o dicionário em dataframe
dfastralCultural = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfastralCultural = resetaIndex(dfastralCultural)
#renomeando colunas
renomeiaColuna(dfastralCultural, 'preco')

### On Line Editora

In [89]:
#filtrando a editora
editoraOnLine = filtrarEditora('On Line Editora')

In [90]:
#verificando os valores duplicados
verificarDuplicados(editoraOnLine, 'isbn')

Total de itens duplicados:  0
Total de itens não duplicados:  1


In [91]:
#filtrando isbn não duplicados
edOnLine = duplicados(editoraOnLine, 'isbn')

In [92]:
#buscando as informações
preco = {}
onLine = 'https://www.editoraonline.com.br/'
#barra de progresso
with tqdm(total=len(edOnLine)) as pbar:
    for item in edOnLine:
        pbar.update(1)
        try:
            navegador.get(onLine)
            sleep(1)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//input[@class="form-control input-lg"]').click()
            #sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@class="form-control input-lg"]')
            digitaCodigo.send_keys(item)
            #clicar em pesquisar
            navegador.find_element_by_xpath('//span[@class="input-group-btn"]').click()
            #entrar nos detalhes
            navegador.find_element_by_xpath('//*[@id="content"]/div[3]/div/div/div[1]/a/img').click()
            sleep(2)
            #pegar preço
            valor = navegador.find_element_by_xpath('//h2[@class="product-price"]')
            preco[item] = valor.text
            
        except:
            preco[item]      = 'Não Encontrado'


In [93]:
#transformando o dicionário em dataframe
dfonLine = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfonLine = resetaIndex(dfonLine)
#renomeando colunas
renomeiaColuna(dfonLine, 'preco')

### Figurati

In [95]:
#filtrando a editora
editoraFigurati = filtrarEditora('Figurati')

In [96]:
#verificando os valores duplicados
verificarDuplicados(editoraFigurati, 'isbn')

Total de itens duplicados:  0
Total de itens não duplicados:  3


In [97]:
#filtrando isbn não duplicados
edFigurati = duplicados(editoraFigurati, 'isbn')

In [98]:
#buscando as informações
preco = {}
figurati = 'https://www.gruponovoseculo.com.br/'
#barra de progresso
with tqdm(total=len(edFigurati)) as pbar:
    for item in edFigurati:
        pbar.update(1)
        try:
            navegador.get(figurati)
            sleep(2)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//div[@class="box only-responsive"]').click()
            sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@class="form-control"]')
            digitaCodigo.send_keys(item)
            #clicar em pesquisar
            navegador.find_element_by_xpath('//button[@class="fas fa-search"]').click()
            sleep(2)
            #entrar nos detalhes
            navegador.find_element_by_xpath('//*[@id="app"]/main/section/div[2]/div[2]/div/article/a[1]').click()
            sleep(1)
            #pegar preço
            valor = navegador.find_element_by_xpath('//div[@class="valor"]')
            preco[item] = valor.text

        except:
            preco[item]      = 'Não Encontrado'


In [99]:
#transformando o dicionário em dataframe
dffigurati = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dffigurati = resetaIndex(dffigurati)
#renomeando colunas
renomeiaColuna(dffigurati, 'preco')

### Verus

In [101]:
#filtrando a editora
editoraVerus = filtrarEditora('Verus')

In [102]:
#verificando os valores duplicados
verificarDuplicados(editoraVerus, 'isbn')

Total de itens duplicados:  21
Total de itens não duplicados:  13


In [103]:
#filtrando isbn não duplicados
edVerus = duplicados(editoraVerus, 'isbn')

In [120]:
#buscando as informações
preco = {}
verus = 'https://www.record.com.br/editoras/verus/'
#barra de progresso
with tqdm(total=len(edVerus)) as pbar:
    for item in edVerus:
        pbar.update(1)
        try:
            navegador.get(verus)
            sleep(3)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//input[@id="dgwt-wcas-search-input-2386"]').click()
            sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@id="dgwt-wcas-search-input-2386"]')
            digitaCodigo.send_keys(item)
            #pesquisar, necessário o enter
            navegador.find_element_by_xpath('//input[@id="dgwt-wcas-search-input-2386"]').send_keys(Keys.ENTER)
            sleep(2)
            #entrar nos detalhes
            navegador.find_element_by_xpath('//span[@class="product_thumbnail_background"]').click()
            sleep(1)
            #pegar preço
            valor = navegador.find_element_by_xpath('//span[@class="woocommerce-Price-amount amount"]')
            preco[item] = valor.text

        except:
            preco[item]      = 'Não Encontrado'


In [123]:
#transformando o dicionário em dataframe
dfverus = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfverus = resetaIndex(dfverus)
#renomeando colunas
renomeiaColuna(dfverus, 'preco')

### Cultrix

In [125]:
#filtrando a editora
editoraCultrix = filtrarEditora('Cultrix')

In [126]:
#verificando os valores duplicados
verificarDuplicados(editoraCultrix, 'isbn')

Total de itens duplicados:  1
Total de itens não duplicados:  3


In [127]:
#filtrando isbn não duplicados
edCultrix = duplicados(editoraCultrix, 'isbn')

In [211]:
#buscando as informações
preco = {}
cultrix = 'https://www.grupopensamento.com.br/vitrine/cultrix'
#barra de progresso
with tqdm(total=len(edCultrix)) as pbar:
    for item in edCultrix:
        pbar.update(1)
        try:
            navegador.get(cultrix)
            sleep(3)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//input[@name="data[Busca][keyword]"]').click()
            #sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@name="data[Busca][keyword]"]')
            digitaCodigo.send_keys(item)
            #pesquisar
            navegador.find_element_by_xpath('//button[@value="Buscar"]').click()
            sleep(2)
            #pegar preço
            valor = navegador.find_element_by_xpath('//*[@id="content"]/nav[2]/ul/li/p')
            preco[item] = valor.text
            
        except:
            preco[item]      = '    Não Encontrado'


In [213]:
#transformando o dicionário em dataframe
dfcultrix = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfcultrix = resetaIndex(dfcultrix)
#renomeando colunas
renomeiaColuna(dfcultrix, 'preco')

In [215]:
dfcultrix['preco'] = dfcultrix['preco'].str[4:]

### Gutenberg

In [217]:
#filtrando a editora
editoraGutenberg = filtrarEditora('Gutenberg')

In [218]:
#verificando os valores duplicados
verificarDuplicados(editoraGutenberg, 'isbn')

Total de itens duplicados:  32
Total de itens não duplicados:  7


In [219]:
#filtrando isbn não duplicados
edGutenberg = duplicados(editoraGutenberg, 'isbn')

In [223]:
#buscando as informações
preco = {}
gutenberg = 'https://grupoautentica.com.br/gutenberg'
#barra de progresso
with tqdm(total=len(edGutenberg)) as pbar:
    for item in edGutenberg:
        pbar.update(1)
        try:
            navegador.get(gutenberg)
            sleep(2)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//*[@id="search_lightbox_open"]').click()
            sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@class="input input--big"]')
            digitaCodigo.send_keys(item)
            #pesquisar
            navegador.find_element_by_xpath('//input[@class="img--icon"]').click()
            sleep(3)
            #entrar nos detalhes
            navegador.find_element_by_xpath('//*[@id="https://grupoautentica.com.br/"]/div[2]/div/div[2]/div/h2/a').click()
            sleep(1)
            #pegar preço
            valor = navegador.find_element_by_xpath('//div[@class="grid__col grid__col--half"]')
            preco[item] = valor.text

        except:
            preco[item]      = 'Não Encontrado'
            

In [225]:
#transformando o dicionário em dataframe
dfgutenberg = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfgutenberg = resetaIndex(dfgutenberg)
#renomeando colunas
renomeiaColuna(dfgutenberg, 'preco')

### Editora Vida

In [227]:
#filtrando a editora
editoraVida = filtrarEditora('Editora Vida')

In [228]:
#verificando os valores duplicados
verificarDuplicados(editoraVida, 'isbn')

Total de itens duplicados:  2
Total de itens não duplicados:  4


In [229]:
#filtrando isbn não duplicados
edVida = duplicados(editoraVida, 'isbn')

In [232]:
#buscando as informações
preco = {}
vida = 'https://editoravida.com.br/'
#barra de progresso
with tqdm(total=len(edVida)) as pbar:
    for item in edVida:
        pbar.update(1)
        try:
            navegador.get(vida)
            sleep(2)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//div[@class="header-cart search-top"]').click()
            sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@id="dgwt-wcas-search-input-1"]')
            digitaCodigo.send_keys(item)
            #pesquisar
            navegador.find_element_by_xpath('//button[@class="dgwt-wcas-search-submit"]').click()
            sleep(3)
            #pegar preço
            valor = navegador.find_element_by_xpath('//span[@class="woocommerce-Price-amount amount"]')
            preco[item] = valor.text

        except:
            preco[item]      = 'Não Encontrado'


In [234]:
#transformando o dicionário em dataframe
dfvida = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfvida = resetaIndex(dfvida)
#renomeando colunas
renomeiaColuna(dfvida, 'preco')

### Cia das Letrinhas

In [236]:
#filtrando a editora
editoraCiaLetrinhas = filtrarEditora('Cia das Letrinhas')

In [237]:
#verificando os valores duplicados
verificarDuplicados(editoraCiaLetrinhas, 'isbn')

Total de itens duplicados:  10
Total de itens não duplicados:  1


In [238]:
#filtrando isbn não duplicados
edCiaLetrinhas = duplicados(editoraCiaLetrinhas, 'isbn')

In [239]:
#buscando as informações
preco = {}
ciaLetrinhas = 'https://www.companhiadasletras.com.br/'
#barra de progresso
with tqdm(total=len(edCiaLetrinhas)) as pbar:
    for item in edCiaLetrinhas:
        pbar.update(1)
        try:
            navegador.get(ciaLetrinhas)
            sleep(2)
            #Clica no campo de busca
            navegador.find_element_by_xpath('/html/body/div[3]/header/a[2]').click()
            sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@id="busca"]')
            digitaCodigo.send_keys(item)
            #pesquisar
            navegador.find_element_by_xpath('//*[@id="bbusca"]/form/table/tbody/tr/td[3]/input').click()
            sleep(3)
            #pegar preço
            valor = navegador.find_element_by_xpath('//div[@class="preco"]')
            preco[item] = valor.text

        except:
            preco[item]      = 'Não Encontrado'


In [241]:
#transformando o dicionário em dataframe
dfciaLetrinhas = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfciaLetrinhas = resetaIndex(dfciaLetrinhas)
#renomeando colunas
renomeiaColuna(dfciaLetrinhas, 'preco')

### Thomas Nelson Brasil

In [243]:
#filtrando a editora
editoraThomasNelson = filtrarEditora('Thomas Nelson Brasil')

In [244]:
#verificando os valores duplicados
verificarDuplicados(editoraThomasNelson, 'isbn')

Total de itens duplicados:  93
Total de itens não duplicados:  5


In [245]:
#filtrando isbn não duplicados
edThomasNelson = duplicados(editoraThomasNelson, 'isbn')

In [246]:
#buscando as informações
preco = {}
thomasNelson = 'https://thomasnelson.com.br/'
#barra de progresso
with tqdm(total=len(edThomasNelson)) as pbar:
    for item in edThomasNelson:
        pbar.update(1)
        try:
            navegador.get(thomasNelson)
            sleep(2)
            #Clica no campo de busca
            navegador.find_element_by_xpath('//input[@id="product-search-keyword"]').click()
            sleep(1)
            #busca através do ISBN
            digitaCodigo = navegador.find_element_by_xpath('//input[@id="product-search-keyword"]')
            digitaCodigo.send_keys(item)
            #pesquisar
            navegador.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/form/div/button').click()
            #detalhes
            navegador.find_element_by_xpath('//*[@id="post-15065"]/div/div[2]/div[1]/h2/a').click()
            sleep(3)
            #leia mais
            navegador.find_element_by_xpath('//a[@id="MoreLink"]').click()
            #pegar preço
            valor = navegador.find_element_by_xpath('//*[@id="tab-additional_information"]/div/table/tbody/tr[9]/td')
            preco[item] = valor.text

        except:
            preco[item]      = 'Não Encontrado'


In [248]:
#transformando o dicionário em dataframe
dfthomasNelson = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfthomasNelson = resetaIndex(dfthomasNelson)
#renomeando colunas
renomeiaColuna(dfthomasNelson, 'preco')

### Editoras:    
L&PM POCKET  
LeYa  
Ática  
Pão Diário  
WMF Martins Fontes   
Martins Fontes  
Quatro Ventos  
Agir  
Elsevier  
Escrituras  
Coquetel

In [250]:
#filtrar apenas as editoras com as quais eu irei extrair do site martins fontes
siteMrtinsFontes = editoras[editoras['editora'].isin(['L&PM POCKET','LeYa','Ática','Pão Diário','WMF Martins Fontes',
                                                    'Martins Fontes','Quatro Ventos','Agir','Elsevier',
                                                    'Escrituras','Coquetel'])]

In [251]:
#verificando os valores duplicados
verificarDuplicados(siteMrtinsFontes, 'isbn')

Total de itens duplicados:  548
Total de itens não duplicados:  42


In [252]:
#filtrando isbn não duplicados
siteMrtinsFontes = duplicados(siteMrtinsFontes, 'isbn')

In [253]:
#shadowRoot
#páginas ocultas dentro do site, acessar o elemento é necessário utilizar javascript
def shadowRootPass(isbnItem):
    navegador.execute_script("return document.querySelector('impulse-autocomplete').shadowRoot.querySelector('input')").send_keys(isbnItem)
    navegador.execute_script("return document.querySelector('impulse-autocomplete').shadowRoot.querySelector('input')").send_keys(Keys.ENTER)
    sleep(5)
    navegador.execute_script('''return document.querySelector('impulse-search').shadowRoot.querySelector('[itemprop="image"] a')''').send_keys(Keys.ENTER)
    sleep(5)

In [274]:
preco = {}

siteMFontes = 'https://www.martinsfontespaulista.com.br/'
navegador.get(siteMFontes)
with tqdm(total=len(siteMrtinsFontes)) as pbar:
    for item in siteMrtinsFontes:
        pbar.update(1)
        try:
            #chama a função do shadow root;
            shadowRootPass(item)
            #há várias opçoes de preço, necessário para pegar o correto.
            if navegador.find_element_by_xpath('//h3[@class="notifymetitle notifyme-title"]').text == 'PRODUTO ESGOTADO NA EDITORA':
                preco[item] = 'NaN'
            elif len(navegador.find_elements_by_xpath('//strong[@class="skuListPrice"]')) > 0:
                preco[item] = navegador.find_element_by_xpath('//strong[@class="skuListPrice"]').text
            else:
                container = navegador.find_element_by_xpath('//*[@id="main"]/section[2]/div/div/div[2]/div[4]/div[1]/div[1]/div/p[1]/em[1]')
                navegador.execute_script("arguments[0].style.display = 'block';", container)
                preco[item] = navegador.find_element_by_xpath('//strong[@class="skuBestPrice"]').text
                     
        except:
            preco[item]  = 'Não Encontrado'
            

In [276]:
#transformando o dicionário em dataframe
dfsiteMFontes = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfsiteMFontes = resetaIndex(dfsiteMFontes)
#renomeando colunas
renomeiaColuna(dfsiteMFontes, 'preco')

In [277]:
#Unindo os dataframes em apenas um:
dfEditoras = pd.concat([dfGente, dftodavia, dfnovoSeculo, dfacademia, dfastralCultural, dfonLine, dffigurati, dfverus,
                        dfcultrix, dfgutenberg, dfvida, dfciaLetrinhas, dfthomasNelson, dfsiteMFontes])

### Adicionando o tipo de capa/acabamento

In [279]:
acabamento = {}

siteMFontes = 'https://www.martinsfontespaulista.com.br/'
navegador.get(siteMFontes)
with tqdm(total=len(dfEditoras['isbn'])) as pbar:
    for item in dfEditoras['isbn']:
        pbar.update(1)
        try:
            shadowRootPass(item)
            #dá foco na ficha técnica para poder pega-lo:
            element = navegador.find_element_by_xpath('//div[@class="product-details-tab"]')
            navegador.execute_script("arguments[0].scrollIntoView(true);", element)
            sleep(5)
            #pega o texto
            acabamento[item] = navegador.find_element_by_xpath('//td[@class="value-field Acabamento"]').text
        except:
            acabamento[item] = 'NaN'

In [281]:
#transformando o dicionário em dataframe
dfAcabamento = dic_dataframe(acabamento, 'acabamento')
#função pra resetar o indice
dfAcabamento = resetaIndex(dfAcabamento)
#renomeando colunas
renomeiaColuna(dfAcabamento, 'acabamento')

### Adicionando informações incompletas

In [264]:
#filtrando Não encontrado nos preços
adPreco = dfEditoras[dfEditoras['preco'].isin(['Não Encontrado'])]

In [285]:
preco = {}

siteMFontes = 'https://www.martinsfontespaulista.com.br/'
navegador.get(siteMFontes)
with tqdm(total=len(adPreco['isbn'])) as pbar:
    for item in adPreco['isbn']:
        pbar.update(1)
        try:
            #chama a função do shadow root;
            shadowRootPass(item)
            
            if navegador.find_element_by_xpath('//h3[@class="notifymetitle notifyme-title"]').text == 'PRODUTO ESGOTADO NA EDITORA':
                preco[item] = 'NaN'
            elif len(navegador.find_elements_by_xpath('//strong[@class="skuListPrice"]')) > 0:
                preco[item] = navegador.find_element_by_xpath('//strong[@class="skuListPrice"]').text
            else:
                container = navegador.find_element_by_xpath('//*[@id="main"]/section[2]/div/div/div[2]/div[4]/div[1]/div[1]/div/p[1]/em[1]')
                navegador.execute_script("arguments[0].style.display = 'block';", container)
                preco[item] = navegador.find_element_by_xpath('//strong[@class="skuBestPrice"]').text
             
            #já conferi todos que caíram no except e não possui cadastro na livraria
        except:
            preco[item] = 'NaN'
            

In [421]:
#transformando o dicionário em dataframe
dfPreco = dic_dataframe(preco, 'preco')
#função pra resetar o indice
dfPreco = resetaIndex(dfPreco)
#renomeando colunas
renomeiaColuna(dfPreco, 'preco')

In [422]:
#transformando as chaves em indice
leftDF  = dfEditoras.set_index('isbn')
rightDF = dfPreco.set_index('isbn')

In [423]:
#incrementa as linhas faltantes do lado esquerdo com as do lado direito
res = leftDF.reindex(columns=leftDF.columns.union(rightDF.columns))

In [424]:
#atualiza os valores da esquerda baseado na direita
res.update(rightDF)

In [425]:
#função pra resetar o indice
df_Editoras = resetaIndex(res)

In [426]:
#adicionando a coluna de acabamento no dataframe
df_Editoras = pd.merge(df_Editoras, dfAcabamento, left_on='isbn', right_on='isbn', how='left')

In [428]:
df_Editoras['preco'] = df_Editoras['preco'].map(lambda x: x.lstrip('R$').rstrip(''))
df_Editoras['preco'] = df_Editoras['preco'].map(lambda x: x.lstrip(' R$').rstrip(''))

In [429]:
df_Editoras['preco'] = df_Editoras['preco'].str.replace(',', '.').astype(float)

In [431]:
df_Editoras.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223 entries, 0 to 222
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   isbn        223 non-null    int64  
 1   preco       176 non-null    float64
 2   acabamento  223 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 7.0+ KB


In [432]:
df_Editoras.to_excel('editorasPerola.xlsx')